In [9]:
#dependencies
#Dependencies
import pandas as pd
import numpy as np
from sportsreference.nba.schedule import Schedule
from datetime import date, datetime

In [15]:
#defining advanced rest

def adv_rest(team,year):
    #creating the rest day calc function

    def rest_days(team,year):

        #creating the team schedule
        schedule = Schedule(team, year)

        dates = []
        results = []
        locations = []
        game_nums = []
        opponents = []



        for game in schedule:

            if game.playoffs == False :
                dates.append(game.datetime)
                results.append(game.result)
                locations.append(game.location)
                game_nums.append(game.game)
                opponents.append(game.opponent_abbr)

        #creating team dictionary
        team_dict = {
            "dates": dates,
            "results": results,
            "locations": locations,
            "game_nums": game_nums,
            "opponent": opponents
        }

        #creating bucks data frame
        team_df = pd.DataFrame(team_dict)
        team_df

        #finding the number of games between each game
        team_df["date_diff"] = 0

        for i in range(1,len(team_df)):    
            team_df["date_diff"][i] = team_df["dates"][i] - team_df["dates"][(i-1)]

        #inserting value for the date diff in the first game
        #using 10 as a standard for the date difference of the first game (most impactful players don't play preseason anyway)
        team_df["date_diff"][0] = 10

        #converting to a time delta
        team_df["date_diff"][0] = pd.to_timedelta(team_df["date_diff"][0], unit = "D")

        #Converting the date difference series back into an integer for easier comparisons
        for day in range(0,len(team_df["date_diff"])):
            team_df["date_diff"][day] = team_df["date_diff"][day] / np.timedelta64(1, 'D')

        #date diff is only the difference between the two games
        #creating a column to represent the "rest" days between the games
        #"rest" in this case is the days between each game where a game is not being played and does not account for practices
        team_df["rest"] = team_df["date_diff"] - 1 

        return team_df
    
    #passing team to the inner function
    team_df = rest_days(team,year)
    
    opp_diff = []
    opp_rest = []
    
    #iterate through the games
    for i in range(0,len(team_df)):
        
        #for each opponent create a rest dataframe
        opp_df = rest_days(str(team_df['opponent'][i]),year)
        
        #append the rest to the lists
        opp_diff.append(opp_df['date_diff'][i])
        opp_rest.append(opp_df['rest'][i])
    
    #new columns for opponents' values
    team_df['opp_date_diff'] = opp_diff
    team_df['opp_rest'] = opp_rest
    
    #creating a rest difference column
    team_df['rest_diff'] = team_df['rest'] - team_df['opp_rest']
    
    return team_df
    

In [16]:
#test1

wolves_df = adv_rest('MIN',2019)

wolves_df.head(20)

C:\Users\Administrator\anaconda3\lib\site-packages\ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Administrator\anaconda3\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Administrator\anaconda3\lib\site-packages\ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Administrator\anaconda3\lib\site-packages\ipykernel_launcher.py:56:

,dates,results,locations,game_nums,opponent,date_diff,rest,opp_date_diff,opp_rest,rest_diff
0,2018-10-17,Loss,Away,1,SAS,10,9,10.0,9.0,0
1,2018-10-19,Win,Home,2,CLE,2,1,2.0,1.0,0
2,2018-10-20,Loss,Away,3,DAL,1,0,2.0,1.0,-1
3,2018-10-22,Win,Home,4,IND,2,1,2.0,1.0,0
4,2018-10-24,Loss,Away,5,TOR,2,1,2.0,1.0,0
5,2018-10-26,Loss,Home,6,MIL,2,1,1.0,0.0,1
6,2018-10-29,Win,Home,7,LAL,3,2,2.0,1.0,1
7,2018-10-31,Win,Home,8,UTA,2,1,2.0,1.0,0
8,2018-11-02,Loss,Away,9,GSW,2,1,2.0,1.0,0
9,2018-11-04,Loss,Away,10,POR,2,1,1.0,0.0,1
